<a href="https://colab.research.google.com/github/julianovale/PO240_Meta_heuristica/blob/main/PO240_Atividade02_Priscila_GA_Inteira_NA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import files
uploaded = files.upload() # subir arquivo Instancia

Saving Instancia10.txt to Instancia10 (2).txt


In [14]:
# importações
%matplotlib inline
# importações
import matplotlib.pyplot as plt
import random 
from random import choice
from random import seed
import numpy as np
import time 
import copy
import imageio
import os
from google.colab import files

def leitura(arquivo):
  arq = open(arquivo, "r") # r- read
  lixo = arq.readline()  # linha com texto Number of jobs
  linha = arq.readline() # linha com número de tarefas
  N = int(linha)         # linha com o número de tarefas
  p = np.zeros(N)  # p = [p0, p1,..., p(N-1)]
  w = np.zeros(N)
  d = np.zeros(N)
  lixo = arq.readline()  # linha com texto Job data 
  for i in range(N):  # repetir N vezes
      # i-ésima linha
      linha = arq.readline().split()  # posições: 0-indice, 1-p, 2-w, 3-d
      p[i] = int(linha[1])
      w[i] = int(linha[2])
      d[i] = int(linha[3])
  return N, p, w, d  

# inicializa população aleatoriamente
# cada individuo da populacao terá o seguinte formato:
# [[individuo], [lista com xi], funojb]
# para cada indivíduo i
# populacao[i][0]  para acessar o indivíduo (codificação real ou inteira)
# populacao[i][1]  para acessar a variável de decisão xi
# populacao[i][2]  para acessar a função-objetivo

def inicializaPopulacao(TP, N):
  populacao = []
  for i in range(TP):   # para cada individuo [lista] da populacao 
      lista = []
      todos = list(range(N))   # lista com todas as tarefas
      for j in range(N):                      # para cada posição do individuo
          aleatorio = choice(todos)           # escolher uma tarefa aleatoria 
          lista.append(aleatorio)             # adicionar à lista
          todos.remove(aleatorio)             # remover da lista auxiliar para não ocorrer repetição
      # alocar espaço para a variável de decisão
      # função-objetivo zerada
      x = [0]*N
      populacao.append([lista, x, 0])   # incluir o individuo completo na populacao
  return populacao

def decodificaXi(populacao, p, N):
  for i in range(len(populacao)):
    for j in range(len(populacao[0])-2):
      for k in range(len(p)):
        populacao[i][1] = [0]*N
        if (k == 0):
          populacao[i][1][k] = 0
        else:
          temp = 0
          for k in range(len(populacao[0][1])-1):
            temp = temp + p[populacao[i][j][k]]
            populacao[i][1][k+1] = temp
  return populacao
        
def calculaFO(decod_xi, p, w, d, M, TP):
  for i in range(TP):
    temp = 0
    for j in range(len(decod_xi[i][0])):
      temp = temp + w[decod_xi[i][0][j]]*(decod_xi[i][1][j]+p[decod_xi[i][0][j]])+M*max(0,decod_xi[i][1][j]+p[decod_xi[i][0][j]]-d[decod_xi[i][0][j]])
      decod_xi[i][2] = temp
  return decod_xi

def torneioBinario(populacao, TP):
  paisSelecionados = []

  todos = list(range(TP))
  for i in range(TP):
    indAleat1 = random.choice(todos)
    indAleat2 = random.choice(todos)
    while (indAleat1 == indAleat2):
      indAleat2 = random.choice(todos)

    if (populacao[indAleat1][2] < populacao[indAleat2][2]):
      paiSelecionado = copy.deepcopy(populacao[indAleat1])
    else:
      paiSelecionado = copy.deepcopy(populacao[indAleat2])

    paisSelecionados.append(paiSelecionado)
  return paisSelecionados

def crossover(paisSelecionados, TP, N):
  # paisSelecionados = [[[0,1,0,1,0], x, fitness]],[],[]]
  PC = 0.95
  filhos = []

  for i in range(TP//2): # 2 pais geram 2 filhos
    doisPais = random.sample(paisSelecionados, 2) 
    aleatorio = random.random() # entre 0 e 1
    if aleatorio <= PC: # crossover acontece
      ponto = random.randint(1, N-1)
      filho1 = doisPais[0][0][:ponto] + [item for item in doisPais[1][0][:] if item not in doisPais[0][0][:ponto]]
      filho2 = doisPais[1][0][:ponto] + [item for item in doisPais[0][0][:] if item not in doisPais[1][0][:ponto]]
    else:
      filho1 = doisPais[0][0]
      filho2 = doisPais[1][0]
    filhos.append([filho1, 0, 0])
    filhos.append([filho2, 0, 0])
  return filhos

def mutacao(filhos, TP, N):
  PM = 0.1
  for i in range(len(filhos)):
    aleatorio = random.random()
    escolha = list(range(N))
    if aleatorio <= PM:
      swap1 = random.choice(escolha)
      swap2 = random.choice(escolha)
      a = filhos[i][0][swap1]
      b = filhos[i][0][swap2]
      filhos[i][0][swap1] = b
      filhos[i][0][swap2] = a
  return filhos

def main():
  # constantes
  TP = 20    # tamanho da população
  G = 50     # número de gerações
  seed() #inicia a semente dos número pseudo randômicos
  
  # leitura
  N, p, w, d = leitura("Instancia10.txt") ### ALTERAR!!!
  
  inicio = time.time()  # começa a contar o tempo
  
  # Inicializa Populacao 
  populacao = inicializaPopulacao(TP, N)

  # Decodifica para obter xi 
  decod_xi = decodificaXi(populacao, p, N)

  # Calcula a função objetivo penalizada  
  M = 50
  populacao = calculaFO(decod_xi, p, w, d, M, TP)


  for g in range(G):
    # seleciona pares de pais aleatoriamente
    paisSelecionados = torneioBinario(populacao, TP)
    
    # crossover
    filhos = crossover(paisSelecionados, TP, N)

    # mutação
    filhos = mutacao(filhos, TP, N)

    # com filhos: Decodifica para obter xi
    filhos_xi = decodificaXi(filhos, p, N)

    # com filhos: Calcula a função objetivo penalizada   
    M = 50
    filhos = calculaFO(filhos_xi, p, w, d, M, TP)

    # populacao próxima geração, seleção: pais e filho
    mista = populacao + filhos 
    mistaOrd = sorted(mista, key=lambda x: x[2]) # se maximização reverse = True
    populacao = mistaOrd[:TP]
  
  print(populacao)
  print(len(populacao))

# fim do GA
  fim = time.time()
  print("Tempo de execução (s): ", fim - inicio)

In [15]:
main()

[[[3, 8, 5, 0, 1, 4, 7, 9, 2, 6], [0, 1.0, 3.0, 5.0, 7.0, 10.0, 13.0, 15.0, 17.0, 20.0], 769.0], [[3, 8, 5, 0, 1, 4, 7, 9, 2, 6], [0, 1.0, 3.0, 5.0, 7.0, 10.0, 13.0, 15.0, 17.0, 20.0], 769.0], [[3, 8, 5, 0, 1, 4, 7, 9, 2, 6], [0, 1.0, 3.0, 5.0, 7.0, 10.0, 13.0, 15.0, 17.0, 20.0], 769.0], [[3, 8, 5, 0, 1, 4, 7, 9, 2, 6], [0, 1.0, 3.0, 5.0, 7.0, 10.0, 13.0, 15.0, 17.0, 20.0], 769.0], [[3, 8, 5, 0, 1, 4, 7, 9, 2, 6], [0, 1.0, 3.0, 5.0, 7.0, 10.0, 13.0, 15.0, 17.0, 20.0], 769.0], [[3, 8, 5, 0, 4, 1, 7, 9, 2, 6], [0, 1.0, 3.0, 5.0, 7.0, 10.0, 13.0, 15.0, 17.0, 20.0], 769.0], [[3, 8, 5, 0, 1, 4, 7, 9, 2, 6], [0, 1.0, 3.0, 5.0, 7.0, 10.0, 13.0, 15.0, 17.0, 20.0], 769.0], [[3, 8, 5, 0, 1, 4, 7, 9, 2, 6], [0, 1.0, 3.0, 5.0, 7.0, 10.0, 13.0, 15.0, 17.0, 20.0], 769.0], [[3, 8, 5, 0, 1, 4, 7, 9, 2, 6], [0, 1.0, 3.0, 5.0, 7.0, 10.0, 13.0, 15.0, 17.0, 20.0], 769.0], [[3, 8, 5, 0, 1, 4, 7, 9, 2, 6], [0, 1.0, 3.0, 5.0, 7.0, 10.0, 13.0, 15.0, 17.0, 20.0], 769.0], [[3, 8, 5, 0, 1, 4, 7, 9, 2, 6], [0, 1.